In [1]:
import os
from dotenv import load_dotenv

MODEL = "mistral"

In [ ]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)


In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

" Sure, here's a classic computer science joke for you:\n\nWhy don't programmers like nature?\n\nBecause it has too many bugs!"

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [6]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Santiago'

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r'C:\Users\addrk\OneDrive\Documents\insurance agent dat\indiafirst-life-insurance-khata-plan-brochure.pdf')
pages = loader.load_and_split()


In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

C:\Users\addrk\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
retriever = vectorstore.as_retriever()
retriever.invoke("TECHNO TRENDS")

[Document(metadata={'source': 'C:\\Users\\addrk\\OneDrive\\Documents\\insurance agent dat\\indiafirst-life-insurance-khata-plan-brochure.pdf', 'page': 7}, page_content='i.e., fr om the date of issuanc e of the policy or the In the sample illustr ation sho wn abo ve, all pr emiums ar e paid at the beginning of the policy y ear and all the \npolicy benefits ar e paid at the end of the policy y ear.'),
 Document(metadata={'source': 'C:\\Users\\addrk\\OneDrive\\Documents\\insurance agent dat\\indiafirst-life-insurance-khata-plan-brochure.pdf', 'page': 10}, page_content='Disclaimer:  IndiaF irst Life Insur ance Compan y Limited, IRDAI Regn No.143, CIN: U66010MH2008PL C1836 79, \nAddr ess: 12th & 13th ﬂoor , North [C] Wing, Tower 4, Nesc o IT Park, Nesc o Center , Western Expr ess High way, \nGoregaon (East), Mumbai – 400 063. Toll free No – 1800 2098 700. IndiaF irst Life Insur ance Compan y Limited is only \nthe name of the Life Insur ance Compan y and IndiaF irst Life “INSURANCE KHA TA” P

In [10]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    "Genrate frequently asked questions based on the document",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: Genrate frequently asked questions based on the document
Answer: 1. Q: What is IndiaFirst Life Insurance Company Limited's IRDAI registration number?
     A: IndiaFirst Life Insurance Company Limited's IRDAI registration number is 143.

2. Q: Where is the head office of IndiaFirst Life Insurance Company Limited located?
   A: The head office of IndiaFirst Life Insurance Company Limited is at 12th & 13th floor, North [C] Wing, Tower 4, Nesc o IT Park, Nesc o Center, Western Express Highway, Goregaon (East), Mumbai – 400 063.

3. Q: How can I contact IndiaFirst Life Insurance Company Limited?
   A: You can contact IndiaFirst Life Insurance Company Limited toll-free at 1800 2098 700.

4. Q: What is the name of the insurance product mentioned in the document?
   A: The insurance product mentioned in the document is "IndiaFirst Life Insurance Company Limited's INSURANCE KHA TA" Plan (Micr o-Insur ance Product (UIN 143N05 7V0 2)).

5. Q: What should I do if I receive spurious or fr

In [ ]:
chain.batch([{"question": q} for q in questions])

In [ ]:
for s in chain.stream({"question": "some more"}):
    print(s, end="", flush=True)